In [8]:
import time
import numpy as np
import yastn
import yastn.tn.mps as mps
import yastn.tn.fpeps as peps
from routines import NSpin12, MapHex127, gates_from_HH, measure_HH

In [9]:
# Hamiltonian
fname = "sherbrooke0v3.pkl"
data = np.load(fname, allow_pickle=True)
HC = data["H"]
HM = {(n,): 1 for n in range(127)}
#
angles = {"p=3": {"beta": (0.50502, 0.35713, 0.19264),
                  "gamma": (-0.14264, -0.26589, -0.34195)},
          "p=4": {"beta": (0.54321, 0.41806, 0.28615, 0.16041),
                  "gamma": (-0.12077, -0.22360, -0.29902, -0.35329)},
          "p=5": {"beta": (0.53822, 0.44776, 0.32923, 0.23056, 0.12587),
                  "gamma": (-0.11764, -0.19946, -0.268736, -0.321586, -0.34583)},}

In [10]:
# group spins
hex = MapHex127()
ops = NSpin12(sym='dense')
#
# translate Hamiltonian to grouped sites
HHC = hex.group_H(HC, ops.z, ops.I)
HHM = hex.group_H(HM, ops.x, ops.I)

In [13]:
problem = "p=5"
D = 8  # PEPS bond dimension
#
betas = angles[problem]['beta']
gammas = angles[problem]['gamma']
#
# initial product state
#
geometry = peps.SquareLattice(dims=hex.dims, boundary='obc')
vectors = {site: ops.vec_x((1,) * Nloc) for site, Nloc in hex.s2N.items()}
psi = peps.product_peps(geometry, vectors)
#
#  run evolution
#
opts_svd_evol = {"D_total": D, "tol": 1e-10}
env_ntu = peps.EnvNTU(psi, which='NN+')
#
infoss = []
t0 = time.time()
for step, (gamma, beta) in enumerate(zip(gammas, betas)):
    gates = gates_from_HH(HHC, gamma)
    infos = peps.evolution_step_(env_ntu, gates, opts_svd=opts_svd_evol, symmetrize=False)
    infoss.append(infos)
    gates = gates_from_HH(HHM, beta)
    infos = peps.evolution_step_(env_ntu, gates, opts_svd=opts_svd_evol, symmetrize=False)
    Delta = peps.accumulated_truncation_error(infoss, statistics='mean')
    print(f"Step: {step}; Rvolution time: {time.time() - t0:3.2f}; Truncation error: {Delta:0.6f}")

Step: 0; Rvolution time: 21.16; Truncation error: 0.000000
Step: 1; Rvolution time: 43.29; Truncation error: 0.000000
Step: 2; Rvolution time: 73.66; Truncation error: 0.000000
Step: 3; Rvolution time: 120.62; Truncation error: 0.008083
Step: 4; Rvolution time: 167.26; Truncation error: 0.028282


In [14]:
opts_svd_ctm = {"D_total": 2 * D}
env_ctm = peps.EnvCTM(psi, init='eye')
t0 = time.time()
for info in env_ctm.ctmrg_(max_sweeps=10, opts_svd=opts_svd_ctm, corner_tol=1e-5, iterator_step=1):
    print("Ctmrg time:", time.time() - t0)
    print(info)

eng = measure_HH(env_ctm, HHC).real
print(f"Energy for {problem}: {eng:0.4f}")

Ctmrg time: 3.504967451095581
CTMRG_out(sweeps=1, max_dsv=nan, converged=False, max_D=16)
Ctmrg time: 94.56372904777527
CTMRG_out(sweeps=2, max_dsv=0.5923716014415269, converged=False, max_D=16)
Ctmrg time: 186.56947779655457
CTMRG_out(sweeps=3, max_dsv=0.012007350262849228, converged=False, max_D=16)
Ctmrg time: 280.8641617298126
CTMRG_out(sweeps=4, max_dsv=0.0003989187690402193, converged=False, max_D=16)
Ctmrg time: 374.5875313282013
CTMRG_out(sweeps=5, max_dsv=5.8191063494979715e-06, converged=True, max_D=16)
Energy for p=5: -145.6594
